# Playground

# Numba

In [ ]:
from src.Sudoku import Sudoku
import numpy as np


grid1 = """000801000
    000000043
    500000000
    000070800
    000000100
    020030000
    600000075
    003400000
    000200600"""

sudoku = Sudoku(grid1)


In [ ]:
from numba import jit, prange

def calculate_candidates(sudoku):

    candidates = np.zeros([9, 9]) 

    for y in range(9):
        for x in range(9):
            if sudoku.get_cell(x, y) != 0:
                candidates[y][x] = 0
                continue

            row = sudoku.get_row(y)
            col = sudoku.get_col(x)
            sq_index = x // 3 + 3 * (y // 3)
            sq = sudoku.get_square(sq_index)
            invalid_numbers = np.concatenate((row, col, sq.flatten()))
            valid_numbers = np.setdiff1d(np.arange(1, 10), invalid_numbers)
            candidates[y][x] = len(valid_numbers)

    return candidates


@jit(nopython=True)
def custom_setdiff1d(ar1, ar2):
    # Create an empty list to hold the result
    result = []
    
    # Convert ar2 to a set for faster lookup
    ar2_set = set(ar2)
    
    # Add elements to result that are in ar1 but not in ar2
    for item in ar1:
        if item not in ar2_set:
            result.append(item)
    
    return np.array(result)

@jit(nopython=True, parallel=True)
def calculate_candidates_2(grid: np.array):

    candidates = np.zeros((9, 9, 9), dtype=np.bool_)

    for y in prange(9):
        for x in prange(9):
            if grid[y, x] != 0:
                continue

            row = grid[y, :]
            col = grid[:, x]
            y0 = (y // 3) * 3
            x0 = (x // 3) * 3
            sq = grid[y0 : y0 + 3, x0 : x0 + 3]
            invalid_numbers = np.concatenate((row, col, sq.flatten()))
            valid_numbers = custom_setdiff1d(np.arange(1, 10), invalid_numbers)
            candidates[y, x, valid_numbers - 1] = 1

    return candidates


In [ ]:
import time

start_time = time.time()

for _ in range(1000000):
    c = calculate_candidates_2(sudoku.grid)

end_time = time.time()

print(f"Time: {end_time - start_time}")


Time: 54.25662636756897


In [ ]:
def decode_candidates(c):
    candidates = []
    for row in c:
        new_row = []
        for elem in row:
            new_row.append([i + 1 for i, e in enumerate(elem) if e])
        candidates.append(new_row)
    
    return candidates

            

In [ ]:
print(sudoku)

- - - 8 - 1 - - -
- - - - - - - 4 3
5 - - - - - - - -
- - - - 7 - 8 - -
- - - - - - 1 - -
- 2 - - 3 - - - -
6 - - - - - - 7 5
- - 3 4 - - - - -
- - - 2 - - 6 - -



In [ ]:
decode_candidates(c)

[[[2, 3, 4, 7, 9],
  [3, 4, 6, 7, 9],
  [2, 4, 6, 7, 9],
  [],
  [2, 4, 5, 6, 9],
  [],
  [2, 5, 7, 9],
  [2, 5, 6, 9],
  [2, 6, 7, 9]],
 [[1, 2, 7, 8, 9],
  [1, 6, 7, 8, 9],
  [1, 2, 6, 7, 8, 9],
  [5, 6, 7, 9],
  [2, 5, 6, 9],
  [2, 5, 6, 7, 9],
  [2, 5, 7, 9],
  [],
  []],
 [[],
  [1, 3, 4, 6, 7, 8, 9],
  [1, 2, 4, 6, 7, 8, 9],
  [3, 6, 7, 9],
  [2, 4, 6, 9],
  [2, 3, 4, 6, 7, 9],
  [2, 7, 9],
  [1, 2, 6, 8, 9],
  [1, 2, 6, 7, 8, 9]],
 [[1, 3, 4, 9],
  [1, 3, 4, 5, 6, 9],
  [1, 4, 5, 6, 9],
  [1, 5, 6, 9],
  [],
  [2, 4, 5, 6, 9],
  [],
  [2, 3, 5, 6, 9],
  [2, 4, 6, 9]],
 [[3, 4, 7, 8, 9],
  [3, 4, 5, 6, 7, 8, 9],
  [4, 5, 6, 7, 8, 9],
  [5, 6, 9],
  [2, 4, 5, 6, 8, 9],
  [2, 4, 5, 6, 8, 9],
  [],
  [2, 3, 5, 6, 9],
  [2, 4, 6, 7, 9]],
 [[1, 4, 7, 8, 9],
  [],
  [1, 4, 5, 6, 7, 8, 9],
  [1, 5, 6, 9],
  [],
  [4, 5, 6, 8, 9],
  [4, 5, 7, 9],
  [5, 6, 9],
  [4, 6, 7, 9]],
 [[],
  [1, 4, 8, 9],
  [1, 2, 4, 8, 9],
  [1, 3, 9],
  [1, 8, 9],
  [3, 8, 9],
  [2, 3, 4, 9],
  [],
  []],
 [[1

# The hardest grid

In [1]:
grid = [
  [9, 0, 0, 0, 8, 0, 0, 0, 5],
  [0, 0, 0, 0, 0, 0, 0, 0, 6],
  [0, 0, 0, 0, 0, 0, 0, 0, 7],
  [0, 8, 0, 0, 0, 5, 0, 0, 4],
  [0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 4, 0, 2, 3, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 3, 0, 0, 0, 0, 0, 2],
  [2, 0, 0, 0, 0, 9, 0, 8, 3]
]

In [5]:
from src.Sudoku import Sudoku
from src.SudokuAnalyzer import SudokuAnalyzer
from src.SudokuScrambler import SudokuScrambler
from src.SudokuSolver import SudokuSolver

sudoku = Sudoku(grid)

In [3]:
analyzer = SudokuAnalyzer(sudoku)
features = analyzer.get_sudoku_description()

In [4]:
from joblib import load
import numpy as np

model = load("sudoku_model.joblib")
model.predict([features])[0]

c:\Users\adria\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


64.45012533877562

In [1]:
grid = [[6, 4, 0, 0, 9, 0, 0, 1, 2],
 [0, 5, 1, 0, 3, 0, 0, 6, 0],
 [8, 0, 0, 0, 0, 0, 7, 0, 0],
 [4, 0, 0, 0, 2, 1, 0, 0, 0],
 [0, 9, 8, 6, 0, 3, 0, 0, 0],
 [1, 0, 2, 0, 0, 0, 6, 0, 3],
 [0, 0, 0, 0, 0, 0, 0, 0, 7],
 [0, 0, 7, 0, 0, 6, 0, 0, 4],
 [3, 0, 0, 0, 0, 7, 9, 8, 6]]

In [2]:
non_zeros = []
for y in range(9):
    for x in range(9):
        if grid[y][x]:
            non_zeros.append((x, y))


In [11]:
from copy import deepcopy

for x, y in non_zeros:
    tmp, grid[y][x] = grid[y][x], 0
    sudoku = Sudoku(tmp_grid)
    solver = SudokuSolver(sudoku)
    result = solver.solve()
    if result is not None:
        print(x, y)
        print(Sudoku(tmp_grid))
        grid = deepcopy(tmp_grid)
    print("Nope")

Nope
Nope
Nope
7 0
6 4 - - 9 - - - 2 
- 5 1 - 3 - - 6 - 
8 - - - - - 7 - - 
4 - - - 2 1 - - - 
- 9 8 6 - 3 - - - 
1 - 2 - - - 6 - 3 
- - - - - - - - 7 
- - 7 - - 6 - - 4 
3 - - - - 7 9 8 6 

